# Prévisions météorologiques en Australie

# Prévision des températures

## Récupération des librairies

In [30]:
# récupération des librairies
import numpy as np
import pandas as pd
import time, datetime
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Modèle de régression

In [31]:
# récupération du jeu de données intermédiaire de prétraitement des données après gestion des valeurs manquantes 
# et création de nouvelles variables et avant sélection raffinée des variables afin d'améliorer la matrice de corrélation
df = pd.read_csv('../data/weatherAUS_not_fully_preprocessed.csv')
df.head()

,Date,Date.1,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Temp_Delta_MinMax,Humidity_Delta
0,2008-12-01,2008-12-01,Albury,13.4,22.9,0.6,2.240000,4.94000,W,44.0,...,1007.7,1007.1,8.000000,7.40000,16.9,21.8,0,0,9.5,-49.0
1,2008-12-02,2008-12-02,Albury,7.4,25.1,0.0,5.472516,7.63054,WNW,44.0,...,1010.6,1007.8,4.431161,4.49925,17.2,24.3,0,0,17.7,-19.0
2,2008-12-03,2008-12-03,Albury,12.9,25.7,0.0,7.440000,7.96000,WSW,46.0,...,1007.6,1008.7,2.200000,2.00000,21.0,23.2,0,0,12.8,-8.0
3,2008-12-04,2008-12-04,Albury,9.2,28.0,0.0,5.472516,7.63054,NE,24.0,...,1017.6,1012.8,4.431161,4.49925,18.1,26.5,0,0,18.8,-29.0
4,2008-12-05,2008-12-05,Albury,17.5,32.3,1.0,7.480000,0.90000,W,41.0,...,1010.8,1006.0,7.000000,8.00000,17.8,29.7,0,0,14.8,-49.0


In [32]:
# affichage des colonnes du DataFrame
df.columns

Index(['Date', 'Date.1', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall',
       'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am',
       'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow', 'Temp_Delta_MinMax',
       'Humidity_Delta'],
      dtype='object')

In [33]:
# indexation du DataFrame par la date
index_date = pd.to_datetime(df['Date'])
df = df.set_index(index_date)
df.drop('Date.1', axis=1, inplace=True)

In [34]:
# création d'un dictionnaire regroupant les observations du DataFrame par ville 
d = dict(tuple(df.groupby('Location')))

In [35]:
# ajout d'une nouvelle clé dans le dictionnaire indiquant la température maximale à J+1 récupérée à partir des valeurs 
# de "MaxTemp" et création d'une liste regroupant l'ensemble des données
place_dfs = []

for place in d:
  d[place]['MaxTempTomorrow'] = d[place].MaxTemp.shift(freq='-1d')
  place_dfs.append(d[place])

In [36]:
# conversion de la liste en DataFrame par concaténation 
df_pre_temp = pd.concat(place_dfs)

In [37]:
# vérification du nombre de valeurs manquantes résultant de l'indisponibilité des observations de "MaxTemp" à J+1 
df_pre_temp.isna().sum()

Date                    0
Location                0
MinTemp                 0
MaxTemp                 0
Rainfall                0
Evaporation             0
Sunshine                0
WindGustDir             0
WindGustSpeed           0
WindDir9am              0
WindDir3pm              0
WindSpeed9am            0
WindSpeed3pm            0
Humidity9am             0
Humidity3pm             0
Pressure9am             0
Pressure3pm             0
Cloud9am                0
Cloud3pm                0
Temp9am                 0
Temp3pm                 0
RainToday               0
RainTomorrow            0
Temp_Delta_MinMax       0
Humidity_Delta          0
MaxTempTomorrow      1505
dtype: int64

In [38]:
# suppression des valeurs manquantes
df_pre_temp.dropna(inplace = True)

In [39]:
# vérification du nombre total de valeurs manquantes
df_pre_temp.isna().sum().sum()

0

In [40]:
# récupération du jeu de données relatif aux coordonnées géographiques des villes australiennes
aus_town_gps = pd.read_csv("../data/aus_town_gps.csv",sep=",")

In [41]:
# récupération du jeu de données relatif au climat australien
aus_climats = pd.read_csv("../data/climatsAUS_v2.csv", sep=";")

In [42]:
# regroupement des climats en 4 catégories: "chaud_humide", "tempéré_froid", "sec" et "méditerranéen" 
climats_type = {'Am':'chaud_humide',
                'Aw':'chaud_humide',
                'Cfa':'chaud_humide',
                'Cfb':'tempéré_froid', 
                'Cfc':'tempéré_froid', 
                'BSh':'sec',
                'BSk':'sec',
                'Bsk':'sec', 
                'Bwh':'sec',
                'Csa':'méditerranéen',
                'Csb':'méditerranéen'              
               }

aus_climats['Clim_type'] = aus_climats['Climat_Koppen'].map(climats_type)

In [43]:
# intégration des coordonnées géographiques et du climat australien dans le DataFrame étudié
df_pre_temp = pd.merge(df_pre_temp, aus_town_gps, how='left', left_on="Location", right_on="Location")
df_pre_temp = pd.merge(df_pre_temp, aus_climats, how='left', left_on="Location", right_on="Location")

In [44]:
# encodage des variables liées au climat
clim_indic = pd.get_dummies(df_pre_temp.Clim_type, prefix='clim')
df_pre_temp = df_pre_temp.join(clim_indic).drop('Clim_type', axis=1)

In [45]:
# ajout de la variable "month" au DataFrame par conversion de la variable "Date" au format datetime
df_pre_temp['month'] = pd.to_datetime(df_pre_temp['Date']).dt.month

In [46]:
# suppression de la variable "Date" désormais inutile
df_pre_temp.drop(columns=['Date'], inplace = True)

In [47]:
# création d'un dictionnaire "seasons_type" regroupant les mois selon les saisons de l'année
seasons_type = {1:'ete',
                2:'ete',
                3:'automne',
                4:'automne', 
                5:'automne', 
                6:'hiver',
                7:'hiver',
                8:'hiver', 
                9:'printemps',
                10:'printemps',
                11:'printemps',
                12:'ete'              
               }

In [48]:
# ajout de la variable catégorielle "Season" au DataFrame à partir du dictionnaire "seasons_type"
df_pre_temp['Season'] = df_pre_temp['month'].map(seasons_type)

In [49]:
# encodage des variables liées à la saison de l'année
season_indic = pd.get_dummies(df_pre_temp.Season, prefix='Season')
df_pre_temp = df_pre_temp.join(season_indic).drop('Season', axis=1)

In [50]:
# création d'un dictionnaire "compass_points" regroupant les directions du vent selon 4 points intercardinaux
compass_points = {'N':'NW',
                'NNW':'NW',
                'WNW':'NW',
                'W':'SW', 
                'WSW':'SW', 
                'SSW':'SW',
                'S':'SE',
                'SSE':'SE', 
                'ESE':'SE',
                'ENE':'NE',
                'E':'NE',
                'NNE':'NE'              
               }

In [51]:
# modification de la variable catégorielle "WindDir3pm" du DataFrame à partir du dictionnaire "compass_points"
df_pre_temp['WindDir3pm'] = df_pre_temp['WindDir3pm'].map(compass_points)

In [52]:
# encodage des variables liées à la direction du vent pendant l'après-midi
winddir_indic = pd.get_dummies(df_pre_temp.WindDir3pm, prefix='WindDir')
df_pre_temp = df_pre_temp.join(winddir_indic).drop('WindDir3pm', axis=1)

In [53]:
# suppression des variables inutiles
df_pre_temp.drop(columns=['WindGustDir', 'WindDir9am', 'Latitude', 
                 'Longitude', 'Climat_Koppen', 'month', 'Location'], inplace = True)

In [54]:
# séparation des données en une matrice de données et en un vecteur cible
target = df_pre_temp.MaxTempTomorrow
data = df_pre_temp.drop('MaxTempTomorrow', axis = 1)

In [55]:
# division des données en un ensemble d'entraînement et un ensemble de test
# de la quantité totale des données disponibles
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

In [56]:
# standardisation des données
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [57]:
# définition du modèle
grad_boost = GradientBoostingRegressor()

In [58]:
start_time = time.time()

# entraînement du modèle
grad_boost.fit(X_train_scaled, y_train)

# prédiction du modèle
y_pred = grad_boost.predict(X_test_scaled)

# underfitting / overfitting
print('Train Data Score: {}'.format(grad_boost.score(X_train_scaled, y_train)))
print('Test Data Score: {}'.format(grad_boost.score(X_test_scaled, y_test)))

# durée d'exécution
gaussian_time = (time.time() - start_time)
print("\n Running Time: %s" % datetime.timedelta(seconds=gaussian_time))

Train Data Score: 0.8325196380384681
Test Data Score: 0.8301899179029455

 Running Time: 0:00:19.989112


## Conclusion

Ce notebook constitue une preuve de concept visant à considérer un modèle de régression pour la prévision d'une variable quantitative à J+1 (ici "MaxTempTomorrow" par l'algorithme du "Gradient Boosting" inspiré de l’algorithme de la descente de gradient).

Les conclusions de cette étude préliminaire sont les suivantes:
- la méthode est concluante et prometteuse
- l'algorithme du "Gradient Boosting" appelé sans argument donne de bons résultats et s'exécute rapidement
- l'algorithme du "Gradient Boosting" fait l'objet d'un très léger sur-apprentissage (score par défaut de la méthode)

Il est possible de poursuivre cette étude par une analyse complémentaire d'autres modèles de régression et par une modélisation des prévisions de température à plus long terme (par exemple à J+3 et J+7).